# Insertion sort

In [3]:
import numpy as np
import random
import copy
import timeit
import pandas as pd
import math as m

In [4]:
def insertion_sort(A):
    for j in range(1, len(A)):
        key = A[j]
        i = j - 1
        while i >= 0 and A[i] > key:
            A[i + 1] = A[i]
            i = i - 1
        A[i + 1] = key

# Merge sort

In [5]:
def merge(A, p, q, r):    
    n1 = q - p + 1
    n2 = r - q
    
    L = [0] * n1
    R = [0] * n2

    for i in list(range(n1)):
        L[i] = A[p + i - 1]
    
    for j in list(range(n2)):
        R[j] = A[q + j]

    L.append(float('inf'))
    R.append(float('inf'))

    i = 1 - 1     # Subtract 1 to adjust to Python indexing
    j = 1 - 1     # Subtract 1 to adjust to Python indexing
    
    for k in list(range(p - 1, r)):     # Subtract 1 from q to adjust to Python range object
        if L[i] <= R[j]:
            A[k] = L[i]
            i = i + 1
        else:
            A[k] = R[j]
            j = j + 1

In [6]:
def merge_alg(A, p, r):
     if p < r:
        q = m.floor((p + r)/2)
        merge_sort(A,p,q)
        merge_sort(A,q-1,p)
        merge(A,p,q,r)

In [7]:
def sort_merge(A):
    p = 1 
    r= len(A) 
    merge_alg(A, p, r)

# Heap sort

In [8]:
def max_heapify(A, i):

    l = 2*i
    r = 2*1+1
    # fra side 152
    if l <= len(A) and A[l] > A[i]:
        largest = l
    else:
        largest = i
    if r <= len(A) and A[r] > A[largest]:
        largest = r
    if largest != i:
        A[i], A[largest] = A[largest], A[i]
        max_heapify(A, largest)
    #fra side 154 

In [9]:
def build_max_heap(A):
    for i in range(len(A)//2, 0, -1):
        max_heapify(A,i)


In [10]:
def heapsort(A):
    A_heap_size = len(A)
    for i in range(A_heap_size, 1, -1):
        A[1], A[i] = A[i], A[1]
        A_heap_size = A_heap_size - 1
        max_heapify(A,1)
        

# Quicksort

In [11]:
def paritition(A, p, r):
    x = A[r]
    i = p-1
    for j in A[p:r]:
        if A[j] <= x:
            i = i+1
            A[i], A[j] = A[j], A[i]
    A[i+1], A[r] = A[r], A[i+1]
    return i+1

In [12]:
def quicksort(A, p, r):
    if p<r:
        q = paritition(A,p,r)
        quicksort(A,p,q-1)
        quicksort(A,q+1,r)

In [13]:
def sort_quick(A):
    p = 1
    r = len(A) -1
    quicksort(A, p, r)

# Data

In [14]:
def create_data(sort_function, n):
    """
    creates lists of arrays with lengths in power of ten, for the best, worst and the average case for the specific sorting algorithm.
    If the sorting algorithm is either merge_sort or quicksort, the p and the r input for
    these are stored for each case as dictionaries.
    
    parameters: 
    The desired sorting function
    n the highest power of ten, deciding how big the biggest arrays shold be.
    """
    np.random.seed(12235)
    best_case = []
    worst_case = []
    average_case = []
    array_lengths = [10**i for i in range(1,n)]
    if sort_function == insertion_sort:
        for length in array_lengths:
            best_case.append(np.array(range(length)))
            worst_case.append(np.flip((range(length))))
            average_case.append(np.random.randint(1,length,length))
    elif sort_function == sort_merge or sort_function == sort_quick:
        for length in array_lengths: 
            best_case.append(np.array(range(length)))
            worst_case.append(np.flip((range(length))))
            #new_array_average = np.random.random(range(length))
            #average_case.append([[new_array_average], new_array_average[0], new_array_average[length-1]])
    elif sort_function == heapsort:
        for length in array_lengths:
            best_case.append(np.array(range(length)))
            worst_case.append(np.flip((range(length))))
            #average_case.append(np.random.random(range(length)))
    else:
        for length in array_lengths:
            best_case.append(np.array(range(length)))
            worst_case.append(np.flip((range(length))))
            #average_case.append(np.random.random(range(length)))   
                                         
    return best_case, worst_case, average_case, array_lengths

# Benchmarking

In [15]:
def benchmarking(sort_function, test_data):
    """
    A function which sorts the test_data based on a specific sort_function and stores the min time used on a number of repetitions.
    
    parameters:
    sort_function = the sorting function used
    test_data = the array to be sorted
    p, q and r = integers which represents indexes in the array to be sorted in the divide-and-conquer approach, None by default.
    
    """
    clock = timeit.Timer(stmt = 'sort_func(copy(data))', globals={'sort_func': sort_function, 'data':test_data, 'copy':copy.copy})

    n_ar, t_ar = clock.autorange()
    t = clock.repeat(repeat=7, number=n_ar)
    time = min(t)
    
    return time

In [16]:
def simulating(case, sort_function, n):
    """
    Function going through benchmarking for best case for a specific algorithm, 
    and saving the results as a file using to_pickle()
    
    parameters:
    desired case as string
    sorting function used
    n the highest power of ten deciding the size of the biggest array to be sorted
    """
    best_case_arrays, worst_case_arrays, average_case_arrays, array_lengths = create_data(sort_function, n)
    data = {}
    i = 0
    if case == 'best case':
        for array in best_case_arrays:
            data[i] = benchmarking(sort_function, array)
            i = i+1
        df_benchmarking = pd.DataFrame(data, columns = (array_lengths), index = ['times'])
        print(data)
        print(df_benchmarking)
        df_benchmarking.to_pickle(path = 'C:/Users/kajsi/Documents/NMBU/Inf221/Semesteroppgave/best_case_sort')
    elif case == 'worst case':
        for array in worst_case_arrays:
            data[i] = benchmarking(sort_function, array)
            i = i+1
        df_benchmarking = pd.DataFrame(data, columns = (array_lengths), index = ['times'])
        print(df_benchmarking)
        df_benchmarking.to_pickle(path = 'C:/Users/kajsi/Documents/NMBU/Inf221/Semesteroppgave/worst_case_sort')
    else:
        for array in average_case_arrays:
            data[i] = benchmarking(sort_function, array)
            i = i+1
        df_benchmarking = pd.DataFrame(data, columns = (array_lengths), index = ['times'])
        print(df_benchmarking)
        df_benchmarking.to_pickle(path = 'C:/Users/kajsi/Documents/NMBU/Inf221/Semesteroppgave/average_case_sort')
    
        

In [17]:
# kjørr

simulating('average case', insertion_sort, 4)


      10   100  1000
times  NaN  NaN  NaN


In [135]:
simulating('best case', sorted, 3)

{0: 0.35211490000074264, 1: 0.2811689999944065}
       10   100
times  NaN  NaN


In [113]:

simulating('best case', sort_merge, 3)


[array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])]


RecursionError: maximum recursion depth exceeded in comparison

In [145]:
simulating('best case', heapsort, 3)

IndexError: index 10 is out of bounds for axis 0 with size 10

In [147]:
simulating('best case', sort_quick, 4) 

{0: 0.2826381000049878, 1: 0.22543669999868143, 2: 0.4660459000006085}
      10   100  1000
times  NaN  NaN  NaN
